In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text
import warnings
warnings.filterwarnings('ignore')

print("🎯 PHASE 2.3: RISK SEGMENTATION & PORTFOLIO INTELLIGENCE")
print("=" * 80)
print("Translating ML insights into actionable business strategies")
print("=" * 80)


🎯 PHASE 2.3: RISK SEGMENTATION & PORTFOLIO INTELLIGENCE
Translating ML insights into actionable business strategies


In [3]:
# Load enhanced dataset and optimization results
df = pd.read_csv('../data/processed/enhanced_features_dataset.csv')
import joblib
optimization_config = joblib.load('../models/business_optimization_config.pkl')
optimal_threshold = optimization_config['optimal_threshold']

print(f"📊 PORTFOLIO INTELLIGENCE SETUP")
print("-" * 50)
print(f"Records: {df.shape[0]:,}")
print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"Default rate: {df['Default'].mean():.2%}")

print(f"\n🎯 ADVANCED RISK SEGMENTATION FRAMEWORK")
print("-" * 60)

# Multi-dimensional risk scoring system
df['financial_stability_score'] = (
    (df['debt_service_coverage_ratio'] / 3 * 0.35) +  # Payment capacity
    (df['employment_stability_index'] * 0.25) +        # Job stability  
    (df['CreditScore'] / 850 * 0.25) +                # Credit quality
    (df['human_capital_score'] * 0.15)                 # Overall profile
)

# Opportunity scoring (profitability potential)
df['profitability_score'] = (
    (df['opportunity_index'] * 0.4) +                  # Base opportunity
    ((df['LoanAmount'] / df['LoanAmount'].max()) * 0.3) +  # Loan size impact
    ((1 - df['InterestRate'] / df['InterestRate'].max()) * 0.3)  # Rate competitiveness
)

# Risk complexity assessment
df['risk_complexity'] = (
    (df['total_stress_flags'] / 4 * 0.4) +            # Warning flags
    (df['probability_default_base'] * 0.35) +          # Default probability
    ((df['payment_shock'] > 0.15).astype(int) * 0.25)  # Payment stress
)

# Create multi-dimensional risk segments
def create_risk_segments(stability, profitability, complexity):
    """Create sophisticated risk segments based on multiple dimensions"""
    
    # Define segment boundaries (quartiles/terciles)
    stability_high = stability > stability.quantile(0.67)
    stability_med = (stability > stability.quantile(0.33)) & (stability <= stability.quantile(0.67))
    
    profit_high = profitability > profitability.quantile(0.67)
    profit_med = (profitability > profitability.quantile(0.33)) & (profitability <= profitability.quantile(0.67))
    
    risk_low = complexity < complexity.quantile(0.33)
    risk_med = (complexity >= complexity.quantile(0.33)) & (complexity < complexity.quantile(0.67))
    
    # Business segment assignment logic
    segments = []
    for i in range(len(stability)):
        if stability_high.iloc[i] and profit_high.iloc[i] and risk_low.iloc[i]:
            segments.append('Premier')  # Best customers
        elif stability_high.iloc[i] and profit_high.iloc[i]:
            segments.append('Prime Plus')  # Very good customers
        elif stability_high.iloc[i] and risk_low.iloc[i]:
            segments.append('Stable Core')  # Safe, steady customers
        elif profit_high.iloc[i] and risk_low.iloc[i]:
            segments.append('Growth Target')  # Profitable opportunities
        elif stability_med.iloc[i] and profit_med.iloc[i]:
            segments.append('Standard')  # Regular business
        elif not risk_low.iloc[i] and not stability_high.iloc[i]:
            segments.append('High Touch')  # Requires manual review
        else:
            segments.append('Selective')  # Case-by-case evaluation
    
    return segments

df['business_segment'] = create_risk_segments(
    df['financial_stability_score'],
    df['profitability_score'], 
    df['risk_complexity']
)

# Segment performance analysis
segment_analysis = df.groupby('business_segment').agg({
    'Default': ['count', 'mean'],
    'LoanAmount': ['mean', 'sum'],
    'InterestRate': 'mean',
    'financial_stability_score': 'mean',
    'profitability_score': 'mean',
    'risk_complexity': 'mean'
}).round(3)

print("📊 BUSINESS SEGMENT PERFORMANCE:")
print(segment_analysis)

# Calculate segment value contribution
df['segment_revenue_potential'] = df['LoanAmount'] * 0.08 * (1 - df['Default'])
df['segment_loss_potential'] = df['LoanAmount'] * 0.65 * df['Default']
df['segment_net_value'] = df['segment_revenue_potential'] - df['segment_loss_potential']

segment_value = df.groupby('business_segment').agg({
    'segment_net_value': ['sum', 'mean'],
    'LoanAmount': 'sum'
}).round(0)

print(f"\n💰 SEGMENT VALUE CONTRIBUTION:")
print(segment_value)


📊 PORTFOLIO INTELLIGENCE SETUP
--------------------------------------------------
Records: 255,347
Optimal threshold: 0.790
Default rate: 11.61%

🎯 ADVANCED RISK SEGMENTATION FRAMEWORK
------------------------------------------------------------
📊 BUSINESS SEGMENT PERFORMANCE:
                 Default         LoanAmount              InterestRate  \
                   count   mean        mean          sum         mean   
business_segment                                                        
Growth Target       7098  0.111  178029.051   1263650205        8.307   
High Touch        120156  0.144  151677.311  18224939030       13.317   
Premier            18193  0.054  125978.655   2291929662        8.324   
Prime Plus         12840  0.057  171936.218   2207661044        8.862   
Selective          29846  0.118   86240.976   2573948173       17.011   
Stable Core        37678  0.083   54369.332   2048527709       15.967   
Standard           29536  0.109  134284.424   3966224749       13

In [5]:
print(f"\n🎯 SEGMENT-SPECIFIC BUSINESS STRATEGIES")
print("-" * 60)

# Define business strategies for each segment
segment_strategies = {
    'Premier': {
        'strategy': 'Fast-track approval with premium rates',
        'approval_threshold': 0.05,  # Very lenient
        'rate_adjustment': -0.5,     # 0.5% rate discount
        'review_level': 'Automated',
        'target_approval_rate': 0.95,
        'business_focus': 'Retention and relationship building'
    },
    'Prime Plus': {
        'strategy': 'Streamlined approval with competitive rates',
        'approval_threshold': 0.15,
        'rate_adjustment': -0.25,
        'review_level': 'Light Review',
        'target_approval_rate': 0.90,
        'business_focus': 'Cross-selling and upselling'
    },
    'Stable Core': {
        'strategy': 'Standard approval process',
        'approval_threshold': optimal_threshold,
        'rate_adjustment': 0.0,
        'review_level': 'Standard',
        'target_approval_rate': 0.80,
        'business_focus': 'Portfolio stability and growth'
    },
    'Growth Target': {
        'strategy': 'Aggressive pursuit with risk monitoring',
        'approval_threshold': 0.25,
        'rate_adjustment': 0.0,
        'review_level': 'Enhanced Due Diligence',
        'target_approval_rate': 0.75,
        'business_focus': 'Revenue growth and market share'
    },
    'Standard': {
        'strategy': 'Standard underwriting with monitoring',
        'approval_threshold': optimal_threshold,
        'rate_adjustment': 0.25,
        'review_level': 'Standard Plus',
        'target_approval_rate': 0.70,
        'business_focus': 'Balanced risk-return optimization'
    },
    'High Touch': {
        'strategy': 'Manual review with enhanced conditions',
        'approval_threshold': 0.40,
        'rate_adjustment': 1.0,
        'review_level': 'Manual Underwriting',
        'target_approval_rate': 0.50,
        'business_focus': 'Risk mitigation and selective approval'
    },
    'Selective': {
        'strategy': 'Conservative approval with high margins',
        'approval_threshold': 0.35,
        'rate_adjustment': 0.75,
        'review_level': 'Senior Review',
        'target_approval_rate': 0.60,
        'business_focus': 'Margin optimization and risk control'
    }
}

# Apply segment-specific strategies
df['segment_strategy'] = df['business_segment'].map(lambda x: segment_strategies[x]['strategy'])
df['segment_threshold'] = df['business_segment'].map(lambda x: segment_strategies[x]['approval_threshold'])
df['rate_adjustment'] = df['business_segment'].map(lambda x: segment_strategies[x]['rate_adjustment'])

# Calculate segment-optimized decisions
model = joblib.load('../models/cost_optimized_model.pkl')

# Prepare features for prediction (same as Phase 2.2)
top_features = [
    'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'DTIRatio',
    'InterestRate', 'LoanTerm', 'NumCreditLines', 'debt_service_coverage_ratio',
    'payment_shock', 'credit_utilization_ratio', 'employment_stability_index',
    'human_capital_score', 'loan_to_income_multiple', 'opportunity_index',
    'total_stress_flags', 'probability_default_base', 'credit_maturity_score',
    'pricing_efficiency_score', 'purpose_risk_score'
]

available_features = [f for f in top_features if f in df.columns]
X_segment = df[available_features].fillna(df[available_features].median())
df['default_probability'] = model.predict_proba(X_segment)[:, 1]

# Apply segment-specific thresholds
df['segment_decision'] = (df['default_probability'] >= df['segment_threshold']).astype(int)
df['adjusted_interest_rate'] = df['InterestRate'] + df['rate_adjustment']

print("📋 SEGMENT STRATEGY IMPLEMENTATION:")
for segment, strategy in segment_strategies.items():
    segment_count = (df['business_segment'] == segment).sum()
    segment_approval = (df[df['business_segment'] == segment]['segment_decision'] == 0).mean()
    print(f"\n{segment} ({segment_count:,} applications):")
    print(f"   Strategy: {strategy['strategy']}")
    print(f"   Threshold: {strategy['approval_threshold']:.3f}")
    print(f"   Actual approval rate: {segment_approval:.1%}")
    print(f"   Target approval rate: {strategy['target_approval_rate']:.1%}")
    print(f"   Review level: {strategy['review_level']}")



🎯 SEGMENT-SPECIFIC BUSINESS STRATEGIES
------------------------------------------------------------
📋 SEGMENT STRATEGY IMPLEMENTATION:

Premier (18,193 applications):
   Strategy: Fast-track approval with premium rates
   Threshold: 0.050
   Actual approval rate: 1.1%
   Target approval rate: 95.0%
   Review level: Automated

Prime Plus (12,840 applications):
   Strategy: Streamlined approval with competitive rates
   Threshold: 0.150
   Actual approval rate: 4.3%
   Target approval rate: 90.0%
   Review level: Light Review

Stable Core (37,678 applications):
   Strategy: Standard approval process
   Threshold: 0.790
   Actual approval rate: 88.1%
   Target approval rate: 80.0%
   Review level: Standard

Growth Target (7,098 applications):
   Strategy: Aggressive pursuit with risk monitoring
   Threshold: 0.250
   Actual approval rate: 6.8%
   Target approval rate: 75.0%
   Review level: Enhanced Due Diligence

Standard (29,536 applications):
   Strategy: Standard underwriting with mo

In [6]:
print(f"\n📊 PORTFOLIO OPTIMIZATION & BALANCE ANALYSIS")
print("-" * 60)

# Portfolio composition analysis
portfolio_composition = df.groupby('business_segment').agg({
    'LoanAmount': ['count', 'sum', 'mean'],
    'Default': 'mean',
    'adjusted_interest_rate': 'mean',
    'segment_decision': lambda x: (x == 0).mean()  # Approval rate
}).round(3)

portfolio_composition.columns = ['Count', 'Total_Exposure', 'Avg_Loan', 'Default_Rate', 'Avg_Rate', 'Approval_Rate']

print("🎯 PORTFOLIO COMPOSITION ANALYSIS:")
print(portfolio_composition)

# Calculate portfolio metrics
total_exposure = df['LoanAmount'].sum()
portfolio_default_rate = df['Default'].mean()
segment_exposure_share = df.groupby('business_segment')['LoanAmount'].sum() / total_exposure

print(f"\n💼 PORTFOLIO BALANCE METRICS:")
print(f"   Total portfolio exposure: ${total_exposure:,.0f}")
print(f"   Overall default rate: {portfolio_default_rate:.2%}")
print(f"   Number of segments: {df['business_segment'].nunique()}")

print(f"\n📈 SEGMENT EXPOSURE DISTRIBUTION:")
for segment, share in segment_exposure_share.sort_values(ascending=False).items():
    print(f"   {segment}: {share:.1%} of total exposure")

# Risk concentration analysis
def calculate_portfolio_risk():
    """Calculate portfolio-level risk metrics"""
    segment_risks = df.groupby('business_segment').agg({
        'Default': 'mean',
        'LoanAmount': 'sum'
    })
    
    # Weighted average default rate by exposure
    weighted_default_rate = (segment_risks['Default'] * segment_risks['LoanAmount']).sum() / segment_risks['LoanAmount'].sum()
    
    # Portfolio diversification (Herfindahl-Hirschman Index)
    exposure_shares = segment_risks['LoanAmount'] / segment_risks['LoanAmount'].sum()
    hhi = (exposure_shares ** 2).sum()
    diversification_score = 1 - hhi
    
    return weighted_default_rate, diversification_score

weighted_default, diversification = calculate_portfolio_risk()

print(f"\n🎲 PORTFOLIO RISK ASSESSMENT:")
print(f"   Exposure-weighted default rate: {weighted_default:.2%}")
print(f"   Portfolio diversification score: {diversification:.3f}")
print(f"   Risk concentration level: {'Low' if diversification > 0.7 else 'Medium' if diversification > 0.5 else 'High'}")



📊 PORTFOLIO OPTIMIZATION & BALANCE ANALYSIS
------------------------------------------------------------
🎯 PORTFOLIO COMPOSITION ANALYSIS:
                   Count  Total_Exposure    Avg_Loan  Default_Rate  Avg_Rate  \
business_segment                                                               
Growth Target       7098      1263650205  178029.051         0.111     8.307   
High Touch        120156     18224939030  151677.311         0.144    14.317   
Premier            18193      2291929662  125978.655         0.054     7.824   
Prime Plus         12840      2207661044  171936.218         0.057     8.612   
Selective          29846      2573948173   86240.976         0.118    17.761   
Stable Core        37678      2048527709   54369.332         0.083    15.967   
Standard           29536      3966224749  134284.424         0.109    14.187   

                  Approval_Rate  
business_segment                 
Growth Target             0.068  
High Touch                0.120  
Pre

In [7]:
print(f"\n🌳 BUSINESS RULE EXTRACTION & DECISION TREES")
print("-" * 60)

# Create interpretable decision tree for business rules
decision_features = [
    'CreditScore', 'DTIRatio', 'employment_stability_index', 
    'debt_service_coverage_ratio', 'total_stress_flags', 'LoanAmount'
]

available_decision_features = [f for f in decision_features if f in df.columns]
X_rules = df[available_decision_features]
y_rules = df['Default']

# Train simple decision tree for rule extraction
rule_tree = DecisionTreeClassifier(
    max_depth=5,  # Keep shallow for interpretability
    min_samples_split=1000,  # Ensure statistical significance
    min_samples_leaf=500,
    random_state=42
)

rule_tree.fit(X_rules, y_rules)

# Extract business rules
business_rules = export_text(rule_tree, feature_names=available_decision_features)

print("📋 EXTRACTED BUSINESS RULES:")
print("=" * 50)
print(business_rules[:2000] + "...")  # Show first part

# Create simplified rule-based segments
def apply_business_rules(row):
    """Apply simplified business rules for segment assignment"""
    credit_score = row['CreditScore']
    dti = row['DTIRatio']
    employment_stability = row['employment_stability_index']
    stress_flags = row['total_stress_flags']
    
    # High-quality segment
    if credit_score >= 720 and dti <= 0.3 and employment_stability >= 0.8:
        return 'Auto-Approve'
    
    # Standard segment with conditions
    elif credit_score >= 650 and dti <= 0.45 and stress_flags <= 1:
        return 'Standard-Review'
    
    # Enhanced review segment
    elif credit_score >= 580 and stress_flags <= 2:
        return 'Enhanced-Review'
    
    # High-risk segment
    else:
        return 'High-Risk-Review'

df['rule_based_segment'] = df.apply(apply_business_rules, axis=1)

# Compare rule-based vs ML-based segmentation
rule_performance = df.groupby('rule_based_segment').agg({
    'Default': ['count', 'mean'],
    'business_segment': lambda x: x.mode()[0] if len(x) > 0 else 'Unknown'
}).round(3)

print(f"\n📊 RULE-BASED SEGMENT PERFORMANCE:")
print(rule_performance)



🌳 BUSINESS RULE EXTRACTION & DECISION TREES
------------------------------------------------------------
📋 EXTRACTED BUSINESS RULES:
|--- debt_service_coverage_ratio <= 0.47
|   |--- debt_service_coverage_ratio <= 0.17
|   |   |--- debt_service_coverage_ratio <= 0.09
|   |   |   |--- class: 0
|   |   |--- debt_service_coverage_ratio >  0.09
|   |   |   |--- employment_stability_index <= 0.99
|   |   |   |   |--- LoanAmount <= 191560.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- LoanAmount >  191560.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- employment_stability_index >  0.99
|   |   |   |   |--- LoanAmount <= 152743.00
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- LoanAmount >  152743.00
|   |   |   |   |   |--- class: 0
|   |--- debt_service_coverage_ratio >  0.17
|   |   |--- employment_stability_index <= 0.99
|   |   |   |--- debt_service_coverage_ratio <= 0.28
|   |   |   |   |--- LoanAmount <= 199050.00
|   |   |   |   |   |--- class: 0
|   |   |  

In [9]:
print(f"\n📈 SEGMENT-SPECIFIC KPIs & MONITORING FRAMEWORK")
print("-" * 60)

# Define KPIs for each segment
segment_kpis = {}

for segment in df['business_segment'].unique():
    segment_data = df[df['business_segment'] == segment]
    
    kpis = {
        'portfolio_share': len(segment_data) / len(df),
        'default_rate': segment_data['Default'].mean(),
        'avg_loan_size': segment_data['LoanAmount'].mean(),
        'approval_rate': (segment_data['segment_decision'] == 0).mean(),
        'profitability': segment_data['segment_net_value'].mean(),
        'risk_score': segment_data['risk_complexity'].mean(),
        'stability_score': segment_data['financial_stability_score'].mean()
    }
    
    segment_kpis[segment] = kpis

# Create KPI dashboard data
kpi_df = pd.DataFrame(segment_kpis).T.round(3)

print("📊 SEGMENT KPI DASHBOARD:")
print(kpi_df)

# Risk monitoring alerts
def generate_risk_alerts(segment_kpis):
    """Generate risk monitoring alerts"""
    alerts = []
    
    for segment, kpis in segment_kpis.items():
        # Alert conditions
        if kpis['default_rate'] > 0.15:
            alerts.append(f"🚨 HIGH DEFAULT RATE in {segment}: {kpis['default_rate']:.1%}")
        
        if kpis['portfolio_share'] > 0.4:
            alerts.append(f"⚠️ HIGH CONCENTRATION in {segment}: {kpis['portfolio_share']:.1%}")
        
        if kpis['profitability'] < 0:
            alerts.append(f"📉 NEGATIVE PROFITABILITY in {segment}: ${kpis['profitability']:.0f}")
    
    return alerts

risk_alerts = generate_risk_alerts(segment_kpis)

print(f"\n🔔 RISK MONITORING ALERTS:")
if risk_alerts:
    for alert in risk_alerts:
        print(f"   {alert}")
else:
    print("   ✅ No critical risk alerts")

# Performance benchmarks
benchmarks = {
    'portfolio_default_rate': 0.12,
    'approval_rate': 0.75,
    'portfolio_diversification': 0.6,
    'profitability_per_loan': 2000
}

actual_performance = {
    'portfolio_default_rate': df['Default'].mean(),
    'approval_rate': (df['segment_decision'] == 0).mean(),
    'portfolio_diversification': diversification,
    'profitability_per_loan': df['segment_net_value'].mean()
}

print(f"\n🎯 PERFORMANCE vs BENCHMARKS:")
for metric, target in benchmarks.items():
    actual = actual_performance[metric]
    status = "✅" if actual >= target else "❌"
    print(f"   {metric}: {actual:.3f} vs {target:.3f} {status}")



📈 SEGMENT-SPECIFIC KPIs & MONITORING FRAMEWORK
------------------------------------------------------------
📊 SEGMENT KPI DASHBOARD:
               portfolio_share  default_rate  avg_loan_size  approval_rate  \
Stable Core              0.148         0.083      54369.332          0.881   
High Touch               0.471         0.144     151677.311          0.120   
Selective                0.117         0.118      86240.976          0.114   
Premier                  0.071         0.054     125978.655          0.011   
Standard                 0.116         0.109     134284.424          0.788   
Prime Plus               0.050         0.057     171936.218          0.043   
Growth Target            0.028         0.111     178029.051          0.068   

               profitability  risk_score  stability_score  
Stable Core          498.315       0.350            0.790  
High Touch         -4753.874       0.706            0.522  
Selective          -1593.724       0.563            0.643  
P

In [11]:
print(f"\n🎯 STRATEGIC RECOMMENDATIONS & ACTION PLAN")
print("-" * 60)

# Generate strategic recommendations based on segment analysis
def generate_strategic_recommendations():
    recommendations = []
    
    # Analyze segment performance
    segment_perf = df.groupby('business_segment').agg({
        'Default': 'mean',
        'segment_net_value': 'mean',
        'LoanAmount': ['count', 'sum']
    })
    
    # High-value segments to grow
    high_value_segments = segment_perf[segment_perf[('segment_net_value', 'mean')] > 2000].index
    for segment in high_value_segments:
        count = segment_perf.loc[segment, ('LoanAmount', 'count')]
        recommendations.append({
            'type': 'Growth Opportunity',
            'segment': segment,
            'action': f"Increase marketing spend to grow {segment} segment",
            'rationale': f"High profitability with {count:,} current applications",
            'priority': 'High'
        })
    
    # High-risk segments to manage
    high_risk_segments = segment_perf[segment_perf[('Default', 'mean')] > 0.15].index
    for segment in high_risk_segments:
        default_rate = segment_perf.loc[segment, ('Default', 'mean')]
        recommendations.append({
            'type': 'Risk Management',
            'segment': segment,
            'action': f"Tighten underwriting criteria for {segment}",
            'rationale': f"Default rate of {default_rate:.1%} exceeds acceptable threshold",
            'priority': 'Medium'
        })
    
    return recommendations

recommendations = generate_strategic_recommendations()

print("💡 STRATEGIC RECOMMENDATIONS:")
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec['type']} - {rec['priority']} Priority")
    print(f"   Segment: {rec['segment']}")
    print(f"   Action: {rec['action']}")
    print(f"   Rationale: {rec['rationale']}")

# Executive summary metrics
executive_summary = {
    'total_applications': len(df),
    'total_exposure': df['LoanAmount'].sum(),
    'portfolio_segments': df['business_segment'].nunique(),
    'optimized_approval_rate': (df['segment_decision'] == 0).mean(),
    'projected_net_profit': df['segment_net_value'].sum(),
    'risk_diversification': diversification,
    'strategic_recommendations': len(recommendations)
}

print(f"\n📊 EXECUTIVE PORTFOLIO SUMMARY:")
print("=" * 60)
print(f"📈 Portfolio Scale:")
print(f"   • Total applications analyzed: {executive_summary['total_applications']:,}")
print(f"   • Total loan exposure: ${executive_summary['total_exposure']:,.0f}")
print(f"   • Business segments identified: {executive_summary['portfolio_segments']}")

print(f"\n🎯 Performance Optimization:")
print(f"   • Segment-optimized approval rate: {executive_summary['optimized_approval_rate']:.1%}")
print(f"   • Projected portfolio net profit: ${executive_summary['projected_net_profit']:,.0f}")
print(f"   • Portfolio risk diversification: {executive_summary['risk_diversification']:.1%}")

print(f"\n💼 Strategic Actions:")
print(f"   • Strategic recommendations generated: {executive_summary['strategic_recommendations']}")
print(f"   • Segment-specific strategies implemented: {len(segment_strategies)}")
print(f"   • Risk monitoring framework established: ✅")

# Save segmentation results
df.to_csv('../data/processed/portfolio_segmented_dataset.csv', index=False)
joblib.dump(segment_strategies, '../models/segment_strategies.pkl')

print(f"\n✅ PHASE 2.3 COMPLETE!")
print("=" * 80)
print("✅ Advanced multi-dimensional risk segmentation implemented")
print("✅ Segment-specific business strategies developed")
print("✅ Portfolio optimization and balance analysis completed")
print("✅ Business rule extraction and decision trees created")
print("✅ KPI monitoring framework established")
print("✅ Strategic recommendations and action plan generated")
print("\n🚀 READY FOR Phase 2.4: Model Interpretability & Regulatory Compliance")



🎯 STRATEGIC RECOMMENDATIONS & ACTION PLAN
------------------------------------------------------------
💡 STRATEGIC RECOMMENDATIONS:

1. Growth Opportunity - High Priority
   Segment: Premier
   Action: Increase marketing spend to grow Premier segment
   Rationale: High profitability with 18,193 current applications

2. Growth Opportunity - High Priority
   Segment: Prime Plus
   Action: Increase marketing spend to grow Prime Plus segment
   Rationale: High profitability with 12,840 current applications

📊 EXECUTIVE PORTFOLIO SUMMARY:
📈 Portfolio Scale:
   • Total applications analyzed: 255,347
   • Total loan exposure: $32,576,880,572
   • Business segments identified: 7

🎯 Performance Optimization:
   • Segment-optimized approval rate: 29.6%
   • Projected portfolio net profit: $-522,127,702
   • Portfolio risk diversification: 65.1%

💼 Strategic Actions:
   • Strategic recommendations generated: 2
   • Segment-specific strategies implemented: 7
   • Risk monitoring framework establi